In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataframe
path_df = "Pickles/df.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)
    
# X_train
path_X_train = "Pickles/X_train.pickle"
with open(path_X_train, 'rb') as data:
    X_train = pickle.load(data)

# X_test
path_X_test = "Pickles/X_test.pickle"
with open(path_X_test, 'rb') as data:
    X_test = pickle.load(data)

# y_train
path_y_train = "Pickles/y_train.pickle"
with open(path_y_train, 'rb') as data:
    y_train = pickle.load(data)

# y_test
path_y_test = "Pickles/y_test.pickle"
with open(path_y_test, 'rb') as data:
    y_test = pickle.load(data)

# features_train
path_features_train = "Pickles/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "Pickles/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "Pickles/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "Pickles/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)
    
# SVM Model
path_model = "Pickles/best_svc.pickle"
with open(path_model, 'rb') as data:
    svc_model = pickle.load(data)

# LRC Model
path_model = "Pickles/best_lrc.pickle"
with open(path_model, 'rb') as data:
    lrc_model = pickle.load(data)
    
    
# MNBC Model
path_model = "Pickles/best_mnbc.pickle"
with open(path_model, 'rb') as data:
    mnbc_model = pickle.load(data)
    
# RFC Model
path_model = "Pickles/best_rfc.pickle"
with open(path_model, 'rb') as data:
    rfc_model = pickle.load(data)
    
# GBC Model
path_model = "Pickles/best_gbc.pickle"
with open(path_model, 'rb') as data:
    gbc_model = pickle.load(data)

# KNNC Model
path_model = "Pickles/best_knnc.pickle"
with open(path_model, 'rb') as data:
    knnc_model = pickle.load(data)
    

# Category mapping dictionary
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

category_names = {
    0: 'business',
    1: 'entertainment',
    2: 'politics',
    3: 'sport',
    4: 'tech'
}

In [3]:
svc_predictions = svc_model.predict(features_test)
lrc_predictions = lrc_model.predict(features_test)
knnc_predictions = knnc_model.predict(features_test)
rfc_predictions = rfc_model.predict(features_test)
gbc_predictions = gbc_model.predict(features_test)
mnbc_predictions = mnbc_model.predict(features_test)

In [4]:
# Indexes of the test set
index_X_test = X_test.index

# We get them from the original df
df_test = df.loc[index_X_test]

# Add the predictions
df_test['svc_predictions'] = svc_predictions
df_test['lrc_predictions'] = lrc_predictions
df_test['knnc_predictions'] = knnc_predictions
df_test['rfc_predictions'] = rfc_predictions
df_test['gbc_predictions'] = gbc_predictions
df_test['mnbc_predictions'] = mnbc_predictions

# Clean columns
df_test = df_test[['Content', 'Category', 'Category_Code', 'lrc_predictions','knnc_predictions','mnbc_predictions','gbc_predictions','svc_predictions','rfc_predictions']]

# Decode
# df_test['Category_Predicted'] = df_test['Prediction']
df_test = df_test.replace({'lrc_predictions':category_names})
df_test = df_test.replace({'svc_predictions':category_names})
df_test = df_test.replace({'gbc_predictions':category_names})
df_test = df_test.replace({'rfc_predictions':category_names})
df_test = df_test.replace({'knnc_predictions':category_names})
df_test = df_test.replace({'mnbc_predictions':category_names})

# Clean columns again
df_test = df_test[['Content', 'Category', 'lrc_predictions','knnc_predictions','mnbc_predictions','gbc_predictions','svc_predictions','rfc_predictions']]

In [5]:
df_test.head()

,Content,Category,lrc_predictions,knnc_predictions,mnbc_predictions,gbc_predictions,svc_predictions,rfc_predictions
1691,Ireland call up uncapped Campbell\r\n\r\nUlste...,sport,sport,sport,entertainment,business,business,business
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,tech,tech,tech,business,business,entertainment
477,Egypt and Israel seal trade deal\r\n\r\nIn a s...,business,politics,business,business,politics,business,business
197,Cairn shares up on new oil find\r\n\r\nShares ...,business,entertainment,politics,politics,entertainment,business,politics
475,Saudi NCCI's shares soar\r\n\r\nShares in Saud...,business,tech,entertainment,tech,entertainment,business,entertainment


In [6]:
def output_article(row_article):
    print('Actual Category: %s' %(row_article['Category']))
    print('Predicted Category: %s' %(row_article['Predicted']))
    print('-------------------------------------------')
    print('Text: ')
    print('%s' %(row_article['Content']))

In [7]:
condition = (df_test['Category'] != df_test['lrc_predictions'])
df_misclassified = df_test[condition]
df_misclassified = df_misclassified[['Content','Category','lrc_predictions']]
df_misclassified.head(3)



,Content,Category,lrc_predictions
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,tech
477,Egypt and Israel seal trade deal\r\n\r\nIn a s...,business,politics
197,Cairn shares up on new oil find\r\n\r\nShares ...,business,entertainment


In [8]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

df_misclassified['Predicted'] = df_misclassified['lrc_predictions']
output_article(df_misclassified.loc[list_samples[0]])
output_article(df_misclassified.loc[list_samples[1]])
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: entertainment
Predicted Category: tech
-------------------------------------------
Text: 
Charity single for quake relief

Singers including Sir Cliff Richard and Boy George are recording a charity single to help raise funds for victims of the Asian tsunami.

They hope the song will raise more than Â£2m for the relief fund. The song, titled Grief Never Grows Old and described as a melancholy ballad, was written by radio DJ Mike Read. Former Boyzone singer Ronan Keating may also take part if a studio can be found close to where he is holidaying in Switzerland. Other music stars being approached include Robin and Barry Gibb of The Bee Gees, jazz sensation Jamie Cullum, Chris Rea and Olivia Newton-John.

Sir Cliff has recorded his vocal part in Barbados, while Boy George has recorded his in New York. Read wrote the song before the Boxing Day tragedy but had thought it was too gloomy to release. Now he plans to have it recorded by a collection of pop stars under the name O

In [9]:
condition = (df_test['Category'] != df_test['knnc_predictions'])
df_misclassified = df_test[condition]
df_misclassified = df_misclassified[['Content','Category','knnc_predictions']]
df_misclassified.head(3)



,Content,Category,knnc_predictions
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,tech
197,Cairn shares up on new oil find\r\n\r\nShares ...,business,politics
475,Saudi NCCI's shares soar\r\n\r\nShares in Saud...,business,entertainment


In [10]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

df_misclassified['Predicted'] = df_misclassified['knnc_predictions']
output_article(df_misclassified.loc[list_samples[0]])
output_article(df_misclassified.loc[list_samples[1]])
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: business
Predicted Category: politics
-------------------------------------------
Text: 
S Korean credit card firm rescued

South Korea's largest credit card firm has averted liquidation following a one trillion won ($960m; Â£499m) bail-out.

LG Card had been threatened with collapse because of its huge debts but the firm's creditors and its former parent have stepped in to rescue it. A consortium of creditors and LG Group, a family owned conglomerate, have each put up $480m to stabilise the firm. LG Card has seven million customers and its collapse would have sent shockwaves through the country's economy.

The firm's creditors - which own 99% of LG Card - have been trying to agree a deal to secure its future for several weeks. They took control of the company in January when it avoided bankruptcy only through a $4.5bn bail-out.

They had threatened to delist the company, a move which would have triggered massive debt redemptions and forced the company into bankruptcy,

In [11]:
condition = (df_test['Category'] != df_test['mnbc_predictions'])
df_misclassified = df_test[condition]
df_misclassified = df_misclassified[['Content','Category','mnbc_predictions']]
df_misclassified.head(3)



,Content,Category,mnbc_predictions
1691,Ireland call up uncapped Campbell\r\n\r\nUlste...,sport,entertainment
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,tech
197,Cairn shares up on new oil find\r\n\r\nShares ...,business,politics


In [12]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

df_misclassified['Predicted'] = df_misclassified['mnbc_predictions']
output_article(df_misclassified.loc[list_samples[0]])
output_article(df_misclassified.loc[list_samples[1]])
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: politics
Predicted Category: tech
-------------------------------------------
Text: 
New 'yob' targets to be unveiled

Fifty new areas getting special help to fight anti-social behaviour in England and Wales will be named on Thursday.

Ten areas have already had access to special prosecutors and local experts and the government is now expanding the crackdown to more towns and cities. Details of how many anti-social behaviour orders (Asbos) were used in the last year are also being published. Labour sees nuisance behaviour as a key election issue but critics claim the record is at best patchy. A year ago, ministers launched their anti-social behaviour plan and Thursday's figures offer a progress check. They will say that in the past year more than 2,600 anti-social behaviour orders were issued by the courts - more than double the total used in the previous four years.

Police have also closed 150 crack houses and issued more than 400 dispersal orders, breaking up groups

In [13]:
condition = (df_test['Category'] != df_test['svc_predictions'])
df_misclassified = df_test[condition]
df_misclassified = df_misclassified[['Content','Category','svc_predictions']]
df_misclassified.head(3)



,Content,Category,svc_predictions
1691,Ireland call up uncapped Campbell\r\n\r\nUlste...,sport,business
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,business
887,Halloween writer Debra Hill dies\r\n\r\nScreen...,entertainment,sport


In [14]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

df_misclassified['Predicted'] = df_misclassified['svc_predictions']
output_article(df_misclassified.loc[list_samples[0]])
output_article(df_misclassified.loc[list_samples[1]])
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: tech
Predicted Category: business
-------------------------------------------
Text: 
Blind student 'hears in colour'

A blind student has developed software that turns colours into musical notes so that he can read weather maps.

Victor Wong, a graduate student from Hong Kong studying at Cornell University in New York State, had to read coloured maps of the upper atmosphere as part of his research. To study "space weather" Mr Wong needed to explore minute fluctuations in order to create mathematical models. A number of solutions were tried, including having a colleague describe the maps and attempting to print them in Braille. Mr Wong eventually hit upon the idea of translating individual colours into music, and enlisted the help of a computer graphics specialist and another student to do the programming work.

"The images have three dimensions and I had to find a way of reading them myself," Mr Wong told the BBC News website. "For the sake of my own study - and for th

In [15]:
condition = (df_test['Category'] != df_test['rfc_predictions'])
df_misclassified = df_test[condition]
df_misclassified = df_misclassified[['Content','Category','rfc_predictions']]
df_misclassified.head(3)



,Content,Category,rfc_predictions
1691,Ireland call up uncapped Campbell\r\n\r\nUlste...,sport,business
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,entertainment
197,Cairn shares up on new oil find\r\n\r\nShares ...,business,politics


In [16]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

df_misclassified['Predicted'] = df_misclassified['rfc_predictions']
output_article(df_misclassified.loc[list_samples[0]])
output_article(df_misclassified.loc[list_samples[1]])
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: business
Predicted Category: tech
-------------------------------------------
Text: 
US insurer Marsh cuts 2,500 jobs

Up to 2,500 jobs are to go at US insurance broker Marsh & McLennan in a shake up following bigger-than-expected losses.

The insurer said the cuts were part of a cost-cutting drive, aimed at saving millions of dollars. Marsh posted a $676m (Â£352m) loss for the last three months of 2004, against a $375m (Â£195.3m) profit a year before. It blamed an $850m payout to settle a price-rigging lawsuit, brought by New York attorney general Elliot Spitzer. Under the settlement announced in January, Marsh took a pre-tax charge of $618m in the October-to-December quarter, on top of the $232m charge from the previous quarter. "Clearly 2004 was the most difficult year in MMC's financial history," Marsh chief executive Michael Cherkasky said.

An ongoing restructuring drive at the group also led to a $337m hit in the fourth quarter, the world's biggest insurer said.

In [17]:
condition = (df_test['Category'] != df_test['gbc_predictions'])
df_misclassified = df_test[condition]
df_misclassified = df_misclassified[['Content','Category','gbc_predictions']]
df_misclassified.head(3)



,Content,Category,gbc_predictions
1691,Ireland call up uncapped Campbell\r\n\r\nUlste...,sport,business
1103,Gurkhas to help tsunami victims\r\n\r\nBritain...,politics,business
477,Egypt and Israel seal trade deal\r\n\r\nIn a s...,business,politics


In [18]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples
df_misclassified['Predicted'] = df_misclassified['gbc_predictions']
output_article(df_misclassified.loc[list_samples[0]])
output_article(df_misclassified.loc[list_samples[1]])
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: business
Predicted Category: politics
-------------------------------------------
Text: 
Laura Ashley chief stepping down

Laura Ashley is parting company with its chief executive Ainum Mohd-Saaid.

The clothing and home furnishing retailer said Ms Mohd-Saaid had resigned for personal reasons. Her departure will come into effect on 1 February and follows the departure of co-chief executive Rebecca Navarednam on 1 January. Ms Mohd-Saaid is to be replaced by Lillian Tan, presently a non-executive director of the company and head of a Malaysian retailer.

In a statement issued on Thursday, Laura Ashley thanked Ms Mohd-Saaid for her services to the company. Its shares were down 8.51% to 10.75p in late Thursday morning trading on the London Stock Exchange.

Since 2002, Ms Tan has been managing director and chief executive of Metrojaya, one of the largest retail groups in Malaysia. Laura Ashley, which is due to issue its next trading statement in the next few weeks, has in r

In [19]:
path_tfidf = "Pickles/tfidf.pickle"
with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

In [20]:
punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('punkt')

def create_features_from_text(text):
    
    # Dataframe creation
    
    df = pd.DataFrame(columns=['Content'])
    df.loc[0] = text
    # \r and \n
    df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    # " when quoting text
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    # Lowercasing the text
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    # Saving the lemmatizer into an object
    wordnet_lemmatizer = WordNetLemmatizer()
    nrows = len(df)
    lemmatized_text_list = []
    for row in range(0, nrows):
        # Create an empty list containing lemmatized words
        lemmatized_list = []
        # Save the text and its words into an object
        text = df.loc[row]['Content_Parsed_4']
        text_words = text.split(" ")
        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        # Join the list
        lemmatized_text = " ".join(lemmatized_list)
        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)
    df['Content_Parsed_5'] = lemmatized_text_list
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
    df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
    
    # TF-IDF
    features = tfidf.transform(df).toarray()
    
    return features

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kyasa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kyasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [22]:
def predict_from_text(text, model):
    
    # Predict using the input model
    prediction = model.predict(create_features_from_text(text))[0]
    prediction_proba = model.predict_proba(create_features_from_text(text))[0]
    
    # Return result
    category = get_category_name(prediction)
    
    print("The predicted category is %s." %(category) )
    print("The conditional probability is: %a" %(prediction_proba))

In [23]:
text = """

The center-right party Ciudadanos closed a deal on Wednesday with the support of the conservative Popular Party (PP) to take control of the speaker’s committee in the Andalusian parliament, paving the way for the regional PP leader, Juan Manuel Moreno, to stand as the candidate for premier of the southern Spanish region. The move would see the Socialist Party (PSOE) lose power in the Junta, as the regional government is known, for the first time in 36 years.

Talks in Andalusia have been ongoing since regional polls were held on December 2. The PSOE, led by incumbent premier Susana Díaz, had been expected to win the early elections, but in a shock result the party took the most seats in parliament, 33, but fell well short of a majority of 55. It was their worst result in the region since Spain returned to democracy. The PP came in second, with 26 seats, while Ciudadanos were third with 21. The major surprise was the strong performance of far-right group Vox, which won more than 391,000 votes (10.9%), giving it 12 deputies. The anti-immigration group is the first of its kind to win seats in a Spanish parliament since the end of the Francisco Franco dictatorship. It now holds the key to power in Andalusia, given that its votes, added to those of the PP and Ciudadanos, constitute an absolute majority.

The move would see the Socialist Party lose power in the region for the first time in 36 years

On Thursday, Marta Bosquet of Ciudadanos was voted in as the new speaker of the Andalusian parliament thanks to 59 votes from her party, the PP and Vox. The other candidate, Inmaculada Nieto of Adelante Andalucía, secured 50 votes – from her own party and 33 from the PSOE.

The speaker’s role in the parliament is key for the calling of an investiture vote and for the selection of the candidate for premier.

Officially, the talks as to the make up of a future government have yet to start, but in reality they are well advanced, according to sources from both the PP and Ciudadanos. The leader of the Andalusian PP is banking on being voted into power around January 16 and wants the majority of his Cabinet to be decided “five days before the investiture vote.”

The speaker’s role in the parliament is key for the calling of an investiture vote and for the selection of the candidate for premier

The PP, which was ousted from power by the PSOE in the national government in June, is keen to take the reins of power in Andalusia as soon as possible. The difficulties that Ciudadanos has faced to justify the necessary inclusion of Vox in the talks, has slowed down progress. Rather than align itself with the far right party, the group – which began life in Catalonia in response to the independence drive, but soon launched onto the national stage – had sought a deal with Adelante Andalucía.

Wednesday was a day of intense talks among the parties in a bid to find a solution that would keep everyone happy. But at 9pm last night, Adelante Andalucía announced that it would not be part of “any deal” and that would instead vote for its own candidates to the speaker’s committee in order to “face up to the right wing and the extreme right.”

The PSOE, meanwhile, argues that having won the elections with a seven-seat lead over the PP gives it the legitimacy to aspire to the control of the regional government and the parliament, and to maintain its positions on the speaker’s committee.



"""

In [24]:
predict_from_text(text, lrc_model)
predict_from_text(text, rfc_model)
predict_from_text(text, gbc_model)
predict_from_text(text, mnbc_model)
predict_from_text(text, knnc_model)
predict_from_text(text, svc_model)

The predicted category is sport.
The conditional probability is: array([0.24167863, 0.22824263, 0.13710732, 0.30346857, 0.08950285])
The predicted category is sport.
The conditional probability is: array([0.11529683, 0.16282381, 0.09729167, 0.61365   , 0.0109377 ])
The predicted category is sport.
The conditional probability is: array([0.2236736 , 0.19153893, 0.15640511, 0.31503408, 0.11334827])
The predicted category is sport.
The conditional probability is: array([0.22627373, 0.17182817, 0.18931069, 0.23176823, 0.18081918])
The predicted category is politics.
The conditional probability is: array([0.11111111, 0.11111111, 0.33333333, 0.22222222, 0.22222222])
The predicted category is business.
The conditional probability is: array([0.27308348, 0.27612034, 0.1286262 , 0.26439054, 0.05777944])


In [25]:
# Politics

text = """Disputes have already broken out within the new political alliance that is working to end 36 years of rule by the Socialist Party (PSOE) in Spain’s southern region of Andalusia.

Just hours after the far-right Vox agreed to support the Popular Party (PP)’s candidate to head the region, Juan Manuel Moreno, its demand for a specific regional department for family affairs is running into opposition from Ciudadanos (Citizens), the third party involved in the government change.

These early clashes suggest it could be difficult to export the model to other parts of Spain

The PP and the liberal Ciudadanos have reached their own governing agreement in the wake of an election that undermined the Socialists’ hold on power, but they need support from Vox’s 12 newly elected lawmakers to make it a reality.

Ciudadanos has refused point-blank to meet with Vox representatives, but the PP has struck its own parallel deal with the far right to ensure its support at the investiture debate, which will take place on January 15 and 16.

On Friday morning, Juan Marín of Ciudadanos said that there are no plans for a separate family affairs department within the government structure they have devised with their PP partners, and which does not include Vox.

The reform party has insisted that the Vox-PP deal does not affect them at all, and Ciudadanos’ national leader, Albert Rivera, said on Thursday that Vox “has had to take a step back, correct itself, and sheathe its sword,” alluding to the fact that Vox has dropped some of its most radical demands such as the deportation of 52,000 undocumented migrants.

 Vox national leader Santiago Abascal (c) and Andalusian leader Francisco Serrano (r).
Vox national leader Santiago Abascal (c) and Andalusian leader Francisco Serrano (r). REUTERS
But Vox insists on a family department, and said it will expect loyalty from the PP on this issue. Meanwhile, Ciudadanos has also warned it will not support the PP-Vox proposal to change the historical memory law for a “concord law.”

These early clashes suggest it could be difficult to export the model to other parts of Spain, where local and regional elections are scheduled to be held this year. The PP president, Pablo Casado, said on Thursday that their separate deals with Ciudadanos and Vox in Andalusia are “a preamble of what’s going to happen in May in Spain.”

The PP is anxious to win back power in regions like Valencia, the Balearic Islands, Castilla-La Mancha, Aragón and Cantabria, and to retain it in Madrid, Murcia, La Rioja and Castilla y León.

Parliamentary debate
The PSOE has already digested the fact that it is losing its hold on Spain’s most populated region. “We will conduct a responsible opposition, and remain vigilant about defending self-government and Andalusia’s presence in Spain,” said the party’s parliamentary spokesman, Mario Jiménez.

The Socialists will not be putting forward a candidate, now that the PP nominee has enough support for an absolute majority in the first round. The debates will take place on January 15 and 16, followed by a vote, said Speaker Marta Bosquet.

The sum of the PP, Ciudadanos and Vox votes is four above the 55 required for a majority. The PSOE lost 14 seats at the December 2 election, down to 33. The Podemos-backed Adelante Andalucía has 17 lawmakers.

"""

predict_from_text(text, lrc_model)
predict_from_text(text, rfc_model)
predict_from_text(text, gbc_model)
predict_from_text(text, mnbc_model)
predict_from_text(text, knnc_model)
predict_from_text(text, svc_model)

The predicted category is sport.
The conditional probability is: array([0.24167863, 0.22824263, 0.13710732, 0.30346857, 0.08950285])
The predicted category is sport.
The conditional probability is: array([0.11529683, 0.16282381, 0.09729167, 0.61365   , 0.0109377 ])
The predicted category is sport.
The conditional probability is: array([0.2236736 , 0.19153893, 0.15640511, 0.31503408, 0.11334827])
The predicted category is sport.
The conditional probability is: array([0.22627373, 0.17182817, 0.18931069, 0.23176823, 0.18081918])
The predicted category is politics.
The conditional probability is: array([0.11111111, 0.11111111, 0.33333333, 0.22222222, 0.22222222])
The predicted category is business.
The conditional probability is: array([0.27308348, 0.27612034, 0.1286262 , 0.26439054, 0.05777944])


In [26]:
# Entertainment

text = """
Cádiz is in style: it has just been included in The New York Times’ list of 52 Places to Go in 2019. The recognition comes on the back of acknowledgment from TripAdvisor, which last year ranked Cádiz eighth on its list of European Destinations On The Rise for 2018.

The journalist Andrew Ferren, who wrote about Cádiz for The New York Times’ list, lives in Spain himself and is no stranger to the southern province’s charms. “I am fascinated by its mix of atmospheres,” he says. “You leap from places as sophisticated and cultured as Jerez de la Frontera to wild landscapes that take your breath away, such as the beach in Bolonia or Alcornocales Park. It’s like the Wild West of southern of Spain.”

“Despite the fact that Cádiz was historically a major maritime link between America and Europe, it’s not very well known to the US public and it’s really worth a visit,” he adds. There are three main reasons why The New York Times recommends a trip to Cádiz:

Culinary delights
 Aponiente restaurant in El Puerto de Santa María.
Aponiente restaurant in El Puerto de Santa María.
Suggestions include the new Western-style gastrobar Saja River, recently opened on Santa Elena street, and Código de Barra in La Candelaria Square. Ferren also suggests crossing the bay from the capital to Puerto de Santa María, where Angel León has his three-starred Michelin restaurant offering “a lyrical poem of seafood.” Just 40 km from here, León has another restaurant within the Meliá Sancti Petri hotel called Alevante, which has just been awarded its first Michelin star. For something more casual, there is La Taberna del Chef del Mar, another of the chef’s brands in El Puerto.

To these suggestions, EL VIAJERO adds several of its own, including Restaurante Café Royalty, which opened opened in 1912, and the bookshop café La Clandestina (José del Toro, 23), which serves scrumptious breakfasts. There is also La Candela (Feduchy, 13) for tapas, while an exquisite combo of Andalusian and Norwegian fare is served at Patría restaurant, located on the hillside of Muela in Vejer de la Frontera.

Jerez de la Frontera and its wineries
 Bodegas Lustau, en Jerez de la Frontera (Cádiz).ampliar foto
Bodegas Lustau, en Jerez de la Frontera (Cádiz). NEIL FARRIN GETTY IMAGES
Around 36 km to the north of Cádiz lies Jerez de la Frontera, known for the fortified wines known in English as sherry. Wineries from the region known as Marco de Jerez (comprising Jerez proper, El Puerto and Sanlúcar) preserve their unique 18th and 19th-century atmosphere. Ferren recommends Díez-Mérito, Lustau and Bodegas Tradición. Foodies are told to seek out Lú in Jerez, owned by the chef JuanLu Fernández

The NMAC Montenmedio Foundation
 Vejer de la Frontera.ampliar foto
Vejer de la Frontera. GETTY IMAGES
The NMAC Montenmedio Foundation of contemporary art sits between Barbate and Vejer de la Frontera. It is a private gallery exhibiting works by artists who are asked to come up with projects inspired by the local history and landscape. The center has work by Olafur Eliasson, James Turrell, Marina Abramovic, Pascale Marthine Tayou, Maurizio Cattelan and Santiago Sierra.

EL VIAJERO expands on Ferren’s recommendations with a few of its own:

1.The Cádiz Carnival
 The Cádiz carnival.ampliar foto
The Cádiz carnival.
An unique and fun festival that takes place from February 28 to March 10. In fact it is so unique that it is applying to be included on the World Intangible Cultural Heritage list.

2. Barrio del Pópulo
 The Pópulo neighborhood.ampliar foto
The Pópulo neighborhood. RAQUEL M. CARBONELL GETTY
This is the oldest neighborhood in Cádiz and features an old Roman theater, the old cathedral and stone arches that lead to bohemian bars such as Pay Pay (Silencio, 1), a former brothel that now has live music, dance, improv and stand-up comedy.

3. Cádiz à la Havana
 Cathedral square in Cádiz.ampliar foto
Cathedral square in Cádiz. RAQUEL M. CARBONELL GETTY
Stroll from the colonial-style Mina Square, with its ficus and palm trees, to the Provincial Museum containing Phoenician sarcophagi, to Campo del Sur avenue which was converted into a Havana esplanade for the 2002 James Bond movie Die Another Day.

4. A wealth of history
 Baelo Claudia Roman site in Tarifa (Cádiz).ampliar foto
Baelo Claudia Roman site in Tarifa (Cádiz). KEN WELSH GETTY
Standing on the frontier between two continents, the province of Cádiz has a long and action-packed history, while its capital is one of the oldest cities in Europe. Remnants of yesteryear can be seen at a number of archeological sites, including Baelo Claudia, Carteia and Doña Blanca.

5. Sanlúcar de Barrameda
 Summer beach horse races in Sanlúcar de Barrameda.ampliar foto
Summer beach horse races in Sanlúcar de Barrameda. JUAN CARLOS TORO
Famous for its summer horse racing on the beach as well as for its wineries, this coastal town has been described by journalist Mariló García as a “dazzling city in Cádiz that boasts history, exquisite seafood, a city center with a mix of palaces and wineries as well as the incomparable landscape of Doñana Natural Park.”

6. Coast and mountains
 Olvera, a white village in Cádiz.ampliar foto
Olvera, a white village in Cádiz. RUDI SEBASTIAN GETTY
Cádiz has miles of windswept beaches that make it a perfect haunt for surfers of various descriptions. In less than an hour, however, you can be in the hills visiting quintessentially Andalusian white villages such as Medina Sidonia, Grazalema and Ubrique.

7. The flamenco route
Located in San Fernando, the Peña Flamenca Camarón de la Isla, named after the famous singer, has shows every week and is a good place to tune into some authentic flamenco artistry.

8. Conil de la Frontera
 The beach in Conil de la Frontera.ampliar foto
The beach in Conil de la Frontera. GETTY IMAGES
There are three national parks that stretch along Cádiz’s Atlantic coast – La Breña, Los Alcornocales and el Estrecho – as well as beaches such as Conil de la Frontera which has the best beach bars around for watching the sun go down.

9. Surfing in Tarifa
In the inlets of Los Lances and Valdevaqueros in Tarifa, wind and kitesurfers can skid across the water with a view of Africa in front and El Estrecho national park behind them.

10. The white villages
Nineteen districts in the Cádiz mountains take you through a string of white villages – Alcalá del Valle, Algar, Algodonales, Arcos de la Frontera, Benaocaz, Bornos, El Bosque, Espera, El Gastor, Grazalema, Olvera, Prado del Rey, Puerto Serrano, Setenil de las Bodegas, Torre Alhaquine, Ubrique, Villaluenga del Rosario, Villamartín and Zahara de la Sierra. The villagers still whitewash their homes with lime as they did in the past, normally once a year before the local fiestas.

"""

predict_from_text(text, lrc_model)
predict_from_text(text, rfc_model)
predict_from_text(text, gbc_model)
predict_from_text(text, mnbc_model)
predict_from_text(text, knnc_model)
predict_from_text(text, svc_model)

The predicted category is sport.
The conditional probability is: array([0.24167863, 0.22824263, 0.13710732, 0.30346857, 0.08950285])
The predicted category is sport.
The conditional probability is: array([0.11529683, 0.16282381, 0.09729167, 0.61365   , 0.0109377 ])
The predicted category is sport.
The conditional probability is: array([0.2236736 , 0.19153893, 0.15640511, 0.31503408, 0.11334827])
The predicted category is sport.
The conditional probability is: array([0.22627373, 0.17182817, 0.18931069, 0.23176823, 0.18081918])
The predicted category is politics.
The conditional probability is: array([0.11111111, 0.11111111, 0.33333333, 0.22222222, 0.22222222])
The predicted category is business.
The conditional probability is: array([0.27308348, 0.27612034, 0.1286262 , 0.26439054, 0.05777944])


In [27]:
# Business

text = """
Vodafone España has informed representatives of its employees that it is putting a collective dismissal plan into action that will affect a maximum of 1,200 workers, 24% of its total workforce in Spain of 5,000 people. The layoffs, news of which was broken by EL PAÍS in November, have been justified by the telecoms giant due to the obligation to reduce costs due to a fall in earnings caused by a continuous reduction of prices.

“In the current market climate, demand for services continues to grow exponentially, but this is not the case with prices,” the company stated in a press release. “Nearly 50% of net new customers are associated with low- or medium-cost offers, something that obliges Vodafone to have a cost structure that is prepared to successfully compete in all segments.”

Vodafone added that the current expectations of clients, “who demand an agile, simple and immediate relationship [with their operator],” is prompting the firm to seek “a more simplified organizational model that strengthens coordination and synergies between teams.”

As such, the company continued, it is looking to “reverse the negative trend of the business, strengthen sustainability, protect our capacity to invest and design a more competitive organization that adapts better to what our customers are asking for.”

The operator says that it is sure it can reach a deal with labor unions so that the measures are as painless as possible. The redundancies will likely take effect at the end of February or the beginning of March.

Vodafone has suffered a great deal in the trade war that was sparked by its rivals Movistar and Orange, after the company opted not to buy the rights for Champions League or La Liga soccer matches on the basis of low returns. The strategy has prompted an exodus of clients from their broadband internet, mobile and pay-TV services.

In the first three quarters of 2018, Vodafone has lost 361,000 cellphone lines (70,000 of which were contracts), 134,000 broadband customers, and 108,000 pay-TV subscriptions. The operator has only seen positive numbers in fiber optic internet, where it has put on 84,000 customers.

The operator executed a similar collective dismissal plan (known in Spanish as an ERE) in 2015, reaching a deal with the main labor unions UGT and STC, and which meant the firing of 1,509 workers – 238 fewer than those initially proposed by the company. Vodafone justified those sackings based on the duplication of roles caused by the purchase of rival operator ONO in 2014, and the absorption of the company’s workforce.

Before the acquisition of ONO, Vodafone also executed an ERE in 2013. On that occasion, the company agreed with unions on the firing of 620 employees, the externalization of services, which affected 130 workers, and changes in the conditions of another 150 people.

"""

predict_from_text(text, lrc_model)
predict_from_text(text, rfc_model)
predict_from_text(text, gbc_model)
predict_from_text(text, mnbc_model)
predict_from_text(text, knnc_model)
predict_from_text(text, svc_model)

The predicted category is sport.
The conditional probability is: array([0.24167863, 0.22824263, 0.13710732, 0.30346857, 0.08950285])
The predicted category is sport.
The conditional probability is: array([0.11529683, 0.16282381, 0.09729167, 0.61365   , 0.0109377 ])
The predicted category is sport.
The conditional probability is: array([0.2236736 , 0.19153893, 0.15640511, 0.31503408, 0.11334827])
The predicted category is sport.
The conditional probability is: array([0.22627373, 0.17182817, 0.18931069, 0.23176823, 0.18081918])
The predicted category is politics.
The conditional probability is: array([0.11111111, 0.11111111, 0.33333333, 0.22222222, 0.22222222])
The predicted category is business.
The conditional probability is: array([0.27308348, 0.27612034, 0.1286262 , 0.26439054, 0.05777944])


In [28]:
# Tech

text = """
Elon Musk told the world in late 2017 that Tesla was taking its automotive know-how and applying it to a totally new challenge: self-driving big rigs. But one year later, he placed the Tesla Semi fourth on a list of priorities for the company, behind the upcoming Model Y compact SUV and an electric pickup truck. This week, Daimler executed a move many years in the making by announcing its own big rig (albeit diesel-powered) outfitted with semi-autonomous technology. And others are following suit.

The German automaker also committed to manufacturing the truck this summer, with deliveries scheduled for later this year. It pledged 500 million euros over the next few years to the continued development of an autonomous big rig, and said it has hired hundreds of employees to move the tech forward. And just like it did when it unveiled the prototype version in 2015, Daimler gave us a ride in the truck to get a taste of what the near future of trucking will look like.

 
While there are a few Tesla Semi prototypes on the road now, and a dozen or so big name companies have placed preorders for the trucks, it doesn’t look like a production version is coming any time soon. Tesla still hasn’t said where or exactly when it will build the trucks, and would likely need to raise more money (or sell a hell of a lot more Model 3s) to fund the project.

DAIMLER FIRST SHOWED OFF A PROTOTYPE IN 2015
This has left the door wide open for companies like Daimler, the parent company of Mercedes-Benz. Daimler announced it was working on its own self-driving big rig in 2015 when it showed off a working prototype called the Freightliner Inspiration Truck. The automaker went big, debuting the truck on the Hoover Dam and offering test rides at Las Vegas Motor Speedway. This week at the Consumer Electronics Show, Daimler returned to Las Vegas to make good on its promise with a production version of that prototype truck.

The new Cascadia is not much more advanced than the prototype was in 2015. In fact, the technology is still pretty limited. Daimler says it’s the first Class 8 commercial truck with Level 2 autonomy (referring to the Society of Automotive Engineers’ scale for self-driving definitions), meaning the driver is in control, but is supported heavily by the truck’s technology in certain situations. In that sense, the new Cascadia essentially has the same basic driver assistance technology many modern cars offer, including automatic lane centering, adaptive cruise control, and emergency braking.

 
The Freightliner Inspiration Truck at the event in 2015.
But the new Cascadia is doing this with a limited set of sensors. There’s a forward-facing camera, a forward-facing radar, a second radar sensor on the right side of the truck. That package pales in comparison to the dozens of cameras, ultrasonic sensors, and radars you’d find powering Autopilot, let alone the Tesla Semi, which is supposed to have a beefed-up version of this same sensor suite.

This helps keep costs down, but means the technology is more in line with what you’d find powering something like Nissan’s ProPilot driver assistance feature as opposed to Autopilot, or even Audi’s supposedly Level 3 system, which uses similar tech, but relies on LIDAR as well.

DAIMLER’S TRUCK HAS MORE IN COMMON WITH NISSAN’S PROPILOT SYSTEM THAN TESLA’S AUTOPILOT
Keeping with a theme of less is more, there’s also no camera-based monitoring system in the truck to make sure the driver pays attention while using the Level 2 features. Instead, the Cascadia uses a system similar to the one found in Tesla’s cars.

A sensor in the steering column measures resistance applied to the steering wheel. If the driver takes their hands off the wheel while using the lane centering feature, the instrument cluster will, after about 15 seconds, surface a warning that tells them to place their hands back on the wheel. If the driver doesn’t do that, the warning changes from yellow to red. After another 60 seconds, if the driver still hasn’t put their hands back on the wheel, the truck will come to a stop on the side of the road.

  
 
The new Cascadia is a far cry from a fully autonomous truck, but based on my brief ride, Daimler has refined the technology compared to the prototype version. The prototype swayed on the highway during my two-mile demo ride in 2015, ping-ponging between the lane markers. The new truck, meanwhile, felt locked to the center of the lane during this week’s ride, which followed the same exact route from a few years ago.

A Daimler representative also told me that, while lane centering is on, the driver can even choose where the system places them in the lane. (For example, if a driver is on a tight one-lane highway and wants to avoid clipping oncoming traffic, they could tell the truck to hug the right lane line.) This is another sign that system is maturing from what debuted in 2015, though it’s a small one.

RELATED

This is what it’s like to ride in Daimler’s self-driving semi truck
Daimler promised some other modern technologies are coming the new Cascadia, though none of it was on display in the preproduction trucks being used for the demonstration. The company plans to offer an optional 10-inch touchscreen in the dashboard, and a 12-inch digital cluster behind the steering wheel. The truck will be able to receive over-the-air software updates, too.

The Cascadia won’t be as stuffed with tech as the Tesla Semi, nor is it as sleek. But it will be available later this year. Daimler has argued that bringing automation to trucking will help squeeze better fuel efficiency out of the millions of miles that its big rigs cover every year. It would decrease the toll those miles take on the drivers. Most importantly, it could help reduce the some 4,000 fatalities that result from crashes involving these massive hunks of machinery. If all goes well, we might have a sense by the end of this year of whether any of that is true.
"""

predict_from_text(text)

TypeError: predict_from_text() missing 1 required positional argument: 'model'

In [ ]:
# Sports

text = """
Spain has agreed to host the soccer final of the Copa Libertadores between Argentina teams River Plate and Boca Juniors. The match will take place on December 9 at the Santiago Bernabéu stadium in Madrid.

The final in Madrid is a punch in the soul to all fans of soccer in Argentina

ONLINE SPORTS DAILY OLE

The final was set to take place in Argentina but was suspended twice after fans turned violent. The first time, on November 25, riot police were called in after supporters of River began throwing stones and bottles at the bus carrying Boca players. Pepper spray used to control the crowds ended up affecting members of the soccer team. The players later shared photos of their injuries. The match was rescheduled but again had to be suspended after fans descended into violence.

In view of the insecurity, the South American Football Confederation (Conmebol), which organizes the competition, decided the only solution was to hold the game in a different country. On Thursday, Spanish Prime Minister Pedro Sánchez said Spain would be “willing to organize the Copa Libertadores final between River and Boca” in a message on Twitter, and an hour later Conmebol president Alejandro Domínguez confirmed the game would take place in Madrid.

 Embedded video

Sebastián Lisiecki
@sebalisiecki
 Así fue la llegada de Boca al Monumental. Pésimo la seguridad q los mete entre toda la gente de River, los jugadores no tienen q hacer gestos, tienen q.jugar adentro. La Policía no controló y después tiró gas pimienta. Insólito todo. Quinto mundo

575
7:23 PM - Nov 24, 2018
637 people are talking about this
Twitter Ads info and privacy
This was how Boca arrived at Monumental stadium. The security that got between the all people of River was terrible. Players shouldn't have to make gesture, they should play. The police did not control the situation and then pepper gas was thrown. Unbelievable. Fifth world.

This is the first time a Copa Libertadores game has been played outside the Americas since the competition began in 1960. Domínguez however was positive about the decision: “[Madrid] has the largest Argentine community overseas, Barajas is the airport with the best connection to Latin America, and it is a city with a great soccer culture.”

But the feeling in Argentina has been less optimistic. The national newspaper La Nación wrote that “it represents, in some way, the structural failure of our soccer, which is unable to organize the best final in the history of the [Copa] Libertadores.” Online sports daily Ole agreed: “The final in Madrid is a punch in the soul to all fans of soccer in Argentina. Ten thousands kilometers away, River-Boca is an unhappy attempt to transform the classic into a Champions League event.” According to a survey run by Ole, 76% of respondents think it is bad that the game has been moved to Spain.

Security risk
In a message on Twitter, Sánchez promised that "security forces have extensive experience of these situations and are already working on the necessary deployments to ensure the event is secure." Despite these assurances, there are concerns there is not enough time to properly secure the event. According to sources familiar with the security operations, between four and six weeks are needed to analyze the security needs of high-risk sporting stadiums and the ensuing preparations can take up to eight months. In the case of the Copa Libertadores final, security officials will have just days to prepare.

River and Boca have a long-standing rivalry fueled largely by the class divide between the teams – River attracts wealthy, upper-class supporters while Boca is known for its working-class fans.

Scheduling issues
The final will take place on Sunday, December 9, on the final day of a three-day national holiday in Spain for Constitution Day on December 6. Madrid receives its highest number of visitors in these dates. In 2017, 600,000 visitors came to the capital, and the hotel occupancy rate was over 80%.

 Conmebol president Alejandro Domínguez on Tuesday.
Conmebol president Alejandro Domínguez on Tuesday.
Many details about the game have yet to be revealed, including how tickets will be sold, what system will be used and who can buy tickets. In Argentina, fans of the visiting team are banned from the stadiums.

Conmebol and soccer club representatives began considering destinations for the match on Tuesday including Doha, the capital of Qatar and the host of the 2022 FIFA World Cup, which offered attractive economic incentives and Miami, which was ruled out by the US Soccer Federation.



"""

predict_from_text(text)

In [ ]:
# Weather

text = """
A polar air mass that entered the Iberian peninsula on Wednesday has already caused sharp drops in temperature, but the worst is yet to come, said the national weather service Aemet.

“An episode of intense cold” is forecast for Friday, when the mercury will continue to plummet across much of Spain. Thirty provinces are on yellow alert, the lowest on a three-color scale, except for Guadalajara, where there is an orange advisory in place. Lows of -10 ºC are expected in Parameras de Molina.

Elsewhere, weather stations have recorded -8.2ºC in La Molina (Girona), at an elevation of 1,700 meters, and -6.8ºC in Puerto de Navacerrada (Madrid).

 Almería has rolled out vehicles to deal with wintry road conditions.
Almería has rolled out vehicles to deal with wintry road conditions. DIPUTACIÓN DE ALMERÍA EUROPA PRESS
Aemet spokesman Rubén del Campo said that the cold spell is not out of the ordinary for a month of January, and noted that “we are emerging from a warm December when temperatures were 1.2ºC above average.”

Temperatures have already dipped between six and eight degrees in a matter of hours in some parts of Spain, said Del Campo. And the northerly wind will increase the feeling of cold.

Temperatures on Friday and Saturday will be “very cold, with lows of five to 10 degrees below average in many parts of northern and eastern Spain, and in the Balearics,” he added.

No snow
However, little to no snow is expected “not for lack of cold, but for lack of precipitation, since the air mass is very dry.”

Alerts are in place in Almería, Granada, Jaén, Aragón, Cantabria, Castilla-La Mancha, Castilla y León except for Salamanca and Valladolid, Catalonia save for Tarragona, Madrid, Navarre, the Valencia region, the Balearic Islands, La Rioja, Asturias and Murcia.

On Saturday, the orange warnings will extend to Córdoba, Salamanca, Valladolid, Galicia and Lleida, where lows of -10 ºC are expected in the Vall d’Aran.


"""

predict_from_text(text)

In [ ]:
# Health

text = """
The obesity epidemic has been on the rise for years, with cases nearly tripling since 1975, according to the World Health Organization (WHO). And the outlook for the future is equally bleak – and that includes Spain, a country that is famed for the positive effects of its supposedly Mediterranean diet.

An investigation by the Mar de Barcelona hospital has found that 80% of men and 55% of women will be overweight by 2030. If the current trend continues, the researchers write, within 11 years more than 27 million people in Spain will be overweight.

Being overweight can mean a higher risk of suffering a number of diseases, including diabetes, hypertension, stroke and cancer

The study, published in the Spanish Cardiology Magazine, points out that this epidemic will mean costs of €3 billion for the country’s health system.

The issue, the experts state, is not an esthetic one, but rather a question of health. Being overweight can mean a higher risk of suffering a number of diseases, including diabetes, hypertension, stroke, cancer and other cardiovascular conditions.

Researchers at the Barcelona hospital revised all of the scientific literature published in Spain on epidemiology and the prevalence of obesity and excess weight between 1987 and 2014 in order to come up with an accurate picture of the state of the country’s residents. “We found the trends and we cross-referenced them with data covering the general population from the National Statistics Institute in order to estimate the figures of obesity and excess weight,” explains Álvaro Hernáez, the chief researcher on the project. The results revealed an “alarming future.”

There are currently 25 million people with excess weight, three million more than a decade ago

DR ALBERT GODAY, AUTHOR OF THE STUDY

“There are currently 25 million people with excess weight, three million more than a decade ago,” explains Dr Albert Goday, another author of the study. “In a conservative scenario, if this trend continues, there will be another three million more – 16% more cases – by 2030.”

“In men, excess weight is more usual up to the age of 50,” explains Goday. “From 50 onward, obesity rises among women. These are issues that are intrinsically linked to hormonal metabolism. From a certain age, it is harder for women to control their weight.”

The experts argue that any weight loss, no matter how small, reduces the risk of contracting one of the associated illnesses. “We aren’t going to find the key to this or a vaccination overnight,” Goday adds. “But any minor intervention will help to reduce the risk. You might think that you have made a huge effort and have only lost four kilos when you need to lose many more, but you must bear in mind that those four kilos are already reducing the risk.”

"""

predict_from_text(text)

In [ ]:
# Animal abuse

text = """
Spain’s animal rights party PACMA posted a 38-second video on Twitter on Friday showing a man freeing a fox from a cage, before hunters immediately start shooting at it.

“Hunters shut what appears to be a fox in a cage and let it out only to pepper it with bullets,” says the accompanying text. “Another ‘isolated case’ as the hunting lobby refers to it. Every week, a trickle of ‘isolated cases.’ In fact, they are dangerous psychopaths with a rifle and a license to carry arms.”

 Video insertado

PACMA
✔
@PartidoPACMA
 Cazadores enjaulan a lo que parece ser un zorro y lo liberan solo para acribillarlo a tiros. Otro "caso aislado", de los que habla el lobby de la caza. Cada semana varios "casos aislados".

En realidad, son peligrosos psicópatas con escopeta y permiso de amas. #YoNoDisparo

4.188
10:43 - 4 ene. 2019
7.443 personas están hablando de esto
Información y privacidad de Twitter Ads
At the start of the video, a man teases the caged animal with a stick. When the cage door is opened, the animal makes a run for it, but is shot at by men armed with rifles who are waiting by the cage.

The release of the video, which has had 255,000 views, coincided with the launch of PACMA’s campaign against the start of fox-hunting season in Galicia. “Fox-hunting season in Galicia has started: hunts that hide behind environmental excuses, championships in which the only reason to compete is to kill. The hunters will be entitled to pursue and kill thousands of foxes in the countryside,” states PACMA.

As it notes on its website, PACMA is the only political group that opposes hunting, and it is currently demanding a nationwide ban. “No animal should die under fire,” say the group. “We will fight tirelessly until hunting becomes a crime.”

No animal should die under fire. We will fight tirelessly until hunting becomes a crime

PACMA

The animal rights group is preparing a report to send to the regional government of Galicia against fox hunts. “We are working hard to make it the first Spanish region to assign resources to protecting foxes instead of killing them,” says a source at PACMA.

Last month, a Spanish hunter who was filmed while he chased and tortured a fox was identified by the Civil Guard in the Spanish province of Huesca. The man, aged 35, is facing charges of crimes against wildlife.

And in November, animal rights groups and political parties reacted with indignation over a viral video shot in Cáceres province of 12 hunting dogs falling off a cliff edge, followed by the deer they were attacking.

"""

predict_from_text(text)